In [1]:
import pandas as pd
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from nltk.sentiment import SentimentIntensityAnalyzer

In [2]:
# Load the dataset from XLSX
data = pd.read_excel('/content/chatgpt1-3.xlsx')  
df = data.head(1000)

In [3]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

# Get the stopwords
stopwords = stopwords.words('english')

# Apply stopwords removal
df['clean_text'] = data['Text'].apply(lambda x: ' '.join([word for word in str(x).split() if word.lower() not in stopwords]))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
<ipython-input-3-9f8d72f3d2a9>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['clean_text'] = data['Text'].apply(lambda x: ' '.join([word for word in str(x).split() if word.lower() not in stopwords]))


In [4]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['clean_text'])

In [5]:
# Download the VADER lexicon
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [6]:
# Initialize VADER sentiment analyzer
sid = SentimentIntensityAnalyzer()

In [24]:
data['sentiment'] = data['clean_text'].apply(lambda x: 'positive' if sid.polarity_scores(x)['compound'] > 0
                                              else 'negative' if sid.polarity_scores(x)['compound'] < 0
                                              else 'neutral')

In [25]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data['clean_text'], data['sentiment'], test_size=0.2, random_state=42)

In [26]:
# Feature extraction
vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

In [27]:
# Train the Random Forest classifier
random_forest = RandomForestClassifier(n_estimators=60, random_state=42)
random_forest.fit(X_train_vectorized, y_train)

RandomForestClassifier(n_estimators=60, random_state=42)

In [28]:
# Predict sentiment labels for the test set
y_pred = random_forest.predict(X_test_vectorized)

In [29]:
# Evaluate the model
accuracy = (y_pred == y_test).mean()
print(f"Accuracy: {accuracy}")

Accuracy: 0.8418


In [30]:
new_tweet = "This is a great product! #happy"
clean_new_tweet = ' '.join([word for word in new_tweet.split() if word.lower() not in stopwords])
new_tweet_vector = vectorizer.transform([clean_new_tweet])
predicted_sentiment = random_forest.predict(new_tweet_vector)
print(f"Predicted sentiment: {predicted_sentiment}")

Predicted sentiment: ['positive']
